In [20]:
import json

# 方法一：使用 with 语句自动管理文件的打开和关闭
file_path = 'raw/train.json'
try:
    with open(file_path, 'r', encoding='utf-8') as file:
        # 读取并解析 JSON 数据
        data = json.load(file)
        # 打印读取的数据
        print(data[0])
except FileNotFoundError:
    print(f"文件 {file_path} 未找到。")
except json.JSONDecodeError:
    print(f"文件 {file_path} 不是有效的 JSON 格式。")

{'id': 396, 'content': '没爹的黑孩到处扔', 'output': '没爹的黑孩 | 到处扔 | Racism | hate [END]'}


In [22]:
# 方法一：使用 with 语句自动管理文件的打开和关闭
file_path = 'raw/test1.json'
try:
    with open(file_path, 'r', encoding='utf-8') as file:
        # 读取并解析 JSON 数据
        test_data = json.load(file)
        # 打印读取的数据
        print(test_data[0])
except FileNotFoundError:
    print(f"文件 {file_path} 未找到。")
except json.JSONDecodeError:
    print(f"文件 {file_path} 不是有效的 JSON 格式。")

{'id': 4169, 'content': '飞周兄弟还不认亲？兄弟啊，还袭击？'}


In [8]:
a = []
for d in data:
    a.append(d['output'].split('|')[2].split(',')[0])
set(a)

{' LGBTQ',
 ' LGBTQ ',
 ' Racism',
 ' Racism ',
 ' Region',
 ' Region ',
 ' Sexism',
 ' Sexism ',
 ' non-hate ',
 ' others',
 ' others '}

In [11]:
target_group_list = ['LGBTQ','Racism','Region','Sexism','others','non-hate']

In [15]:
case = {'id': 5219, 'content': '想反女权还得看女权，舔狗龟男理中客们不是男人能打醒的', 'output': '女权 | 想反女权还得看女权 | Sexism | hate [SEP] 舔狗龟男理中客们 | 不是男人能打醒的 | Sexism | hate [END]'}

In [17]:
prompt = '''本次微调任务为细粒度片段级中文仇恨言论识别，基于给定的文本，任务的输入为社交媒体文本，输出为仇恨四元组，顺序依次为评论对象（Target）、论点（Argument）、目标群体（Targeted Group）、是否仇恨（Hateful）。具体说明如下：

评论对象（Target）：帖子的评述对象，如一个人或一个群体。当实例无具体目标时设为NULL。

论点（Argument）：包含对评论目标关键论点的信息片段。

目标群体（Targeted Group）：指包含仇恨信息的评论对象-论点对涉及的目标群体。标注的目标群体包括LGBTQ,Racism,Region,Sexism,others共5类。

是否仇恨（Hateful）：评论对象-论点对是否构成了对某些群体的仇恨言论。有是hate没有是non-hate。对于非仇恨文本和不包含特定群体的一般攻击性言论，设为Non-hate。

由于样本中可能有多个评论对象，因此可以包含多个四元组。 每个四元组中各个元素之间用" | "分割，并利用 [END] 结尾；如果一条样本中包含多个四元组，不同四元组之间利用 [SEP] 分割。

我现在的训练数据是例如这种格式的，这是其中一条样例，content是需要识别的内容，output是输出的仇恨四元组：
{'id': 5219,
 'content': '想反女权还得看女权，舔狗龟男理中客们不是男人能打醒的',
 'output': '女权 | 想反女权还得看女权 | Sexism | hate [SEP] 舔狗龟男理中客们 | 不是男人能打醒的 | Sexism | hate [END]'}

我应该如何设计instruction，把这些数据变成能lora微调出好效果的这种格式{"instruction": "","input": "","output": ""}'''

In [18]:
train_json = []
for d in data:
    tmp_dic = {}
    tmp_dic["instruction"] = '''请从中文社交媒体文本中识别仇恨言论四元组。按以下要求处理：\n1.识别评论对象(Target)，无目标时写NULL\n2.提取对应论点(Argument)\n3.确定目标群体(Targeted Group)：LGBTQ/Racism/Region/Sexism/others\n4.判断是否仇恨(Hateful)：hate/Non-hate\n\n输出格式：Target | Argument | Targeted Group | Hateful [SEP]... [END]\n多个四元组用[SEP]分隔，最后用[END]结尾'''
    tmp_dic["input"] = d['content']
    tmp_dic["output"] = d['output']
    train_json.append(tmp_dic)

In [25]:
test_json = []
for d in test_data:
    tmp_dic = {}
    tmp_dic["instruction"] = '''请从中文社交媒体文本中识别仇恨言论四元组。按以下要求处理：\n1.识别评论对象(Target)，无目标时写NULL\n2.提取对应论点(Argument)\n3.确定目标群体(Targeted Group)：LGBTQ/Racism/Region/Sexism/others\n4.判断是否仇恨(Hateful)：hate/Non-hate\n\n输出格式：Target | Argument | Targeted Group | Hateful [SEP]... [END]\n多个四元组用[SEP]分隔，最后用[END]结尾'''
    tmp_dic['id'] = d['id']
    tmp_dic["input"] = d['content']
    tmp_dic["output"] = ""
    test_json.append(tmp_dic)

In [24]:
# 方法一：使用 with 语句自动管理文件的打开和关闭
file_path = 'train_json/train_json.json'
try:
    with open(file_path, 'w', encoding='utf-8') as file:
        # 将 list 转换为 JSON 格式并写入文件
        json.dump(train_json, file, ensure_ascii=False, indent=4)
    print(f"数据已成功保存到 {file_path}")
except Exception as e:
    print(f"保存文件时出现错误: {e}")

数据已成功保存到 train_json/train_json.json


In [27]:
# 方法一：使用 with 语句自动管理文件的打开和关闭
file_path = 'train_json/test_json.json'
try:
    with open(file_path, 'w', encoding='utf-8') as file:
        # 将 list 转换为 JSON 格式并写入文件
        json.dump(test_json, file, ensure_ascii=False, indent=4)
    print(f"数据已成功保存到 {file_path}")
except Exception as e:
    print(f"保存文件时出现错误: {e}")

数据已成功保存到 train_json/test_json.json


In [28]:
# 方法一：使用 with 语句自动管理文件的打开和关闭
file_path = 'train_json/train_json.json'
try:
    with open(file_path, 'r', encoding='utf-8') as file:
        # 读取并解析 JSON 数据
        train_data = json.load(file)
        # 打印读取的数据
        print(data[0])
except FileNotFoundError:
    print(f"文件 {file_path} 未找到。")
except json.JSONDecodeError:
    print(f"文件 {file_path} 不是有效的 JSON 格式。")

{'id': 396, 'content': '没爹的黑孩到处扔', 'output': '没爹的黑孩 | 到处扔 | Racism | hate [END]'}
